In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORTING LIBRARIES

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('/kaggle/input/top-movies/movies.csv')
df


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

# Split the data into training and testing sets

In [ ]:

X = df['title']  # Movie titles
y = df['genres']  # Movie genres


In [ ]:
X

In [ ]:
y

# REMOVING DUPLICATES AND CONVERTING TO MATRIX 


In [ ]:
# Remove duplicates
df.drop_duplicates(inplace=True)

# Create a user-item rating matrix
user_item_matrix = df.pivot(index='movieId', columns='title', values='genres').fillna(0)
user_item_matrix

In [ ]:
df.dtypes


# training and splitting the model

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Text vectorization using TF-IDF

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# One-hot encoding of genres

In [ ]:

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_encoded = mlb.fit_transform(y_train.str.split('|'))
y_test_encoded = mlb.transform(y_test.str.split('|'))

# Train a Random Forest classifier

In [ ]:

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_vec, y_train_encoded)


# Predict genres for test data

In [ ]:

y_pred_encoded = classifier.predict(X_test_vec)

# Inverse transform to get genre labels

In [ ]:

y_pred_labels = mlb.inverse_transform(y_pred_encoded)

# CALCULATE ACCURACY 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)

# Calculate precision, recall, and F1-score for each genre
precision = precision_score(y_test_encoded, y_pred_encoded, average='micro')
recall = recall_score(y_test_encoded, y_pred_encoded, average='micro')
f1 = f1_score(y_test_encoded, y_pred_encoded, average='micro')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-score: {f1 * 100:.2f}%")
